In [17]:
# basic imports
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization and entity extraction
import spacy
nlp = spacy.load('en_core_web_md')

# plotting and visualization
import altair as alt

import matplotlib.pyplot as plt
%matplotlib inline

# LING 111 Final Project - An Analysis of Shared Keywords in Newspaper Headlines and Articles

It has become common for online news sources to utilize ‘clickbait’ or short, overgeneralized titles in order to grasp the attention of potential readers. Sometimes, the titles don’t accurately reflect the actual content of the article, and in many cases are misleading due to the ways they can be interpreted.


In this project, I analyze the shared keywords between the headlines and article content of a student paper at the University of California, Santa Barbara, and explore the relationship between the amount of information in headlines and reader engagement with the actual article.

---

## Background

The Bottom Line is a student-run, student-funded newspaper at the University of California, Santa Barbara. While the newspaper distributes its physical paper across Isla Vista, California, its digital content can be viewed on its website https://thebottomline.as.ucsb.edu/.

The data of this research project is obtained through The Bottom Line website using WordPress, which the newspaper is hosted on. Each row of the dataset represents an article from the newspaper's 'NEWS' section during the 2020-2021 academic year, with the variables being the article title, written content, and page views.

> **Table 1**: variable descriptions for each variable in the dataset.

Variable | Description
--- | ---
Title | Title of the article; headline.
Content | Written content within the article.
Views | The number of page views of the article as counted by WordPress.



In [18]:
df = pd.read_csv('tbl_news.csv')
df.head()

,Title,Content,Views
0,UC Irvine Offers UC-Wide Environmental Injusti...,UC Irvine (UCI) recently announced it will ope...,616
1,UC Santa Barbara Opens Clinic to Distribute Jo...,As COVID-19 vaccines become more widely availa...,701
2,AS Passes Bill to Reform the Honorarium of Sen...,The UC Santa Barbara (UCSB) Associated Student...,1097
3,Cops Off Campus: A March to Recognize the Move...,"On May 17, UC Santa Barbara’s (UCSB) Cops Off ...",748
4,Wealthy Countries are Hoarding COVID-19 Vaccin...,If there is one thing to be learned from this ...,1458


In [19]:
df.shape

(64, 3)

---

## Exploratory Analysis

### Topic Modeling
NLTK and spaCy is initially used to pre-process the headline and content text by removing stopwords and lemmatizing the text. Then, Gensim's LDA model is used to perform topic modeling across all article content. (TBD: I also repeat this with the headlines, just to see if the same topics appear.)

The code and methods in this exploration is mainly based off of this article: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/.

#### Pre-processing via NLTK and spaCy

Removing punctuation, unnecessary characters, whitespace and tokenizing the content.

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [21]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [22]:
# Convert to list
data = df.Content.values.tolist()

data_words = list(sent_to_words(data))

print(data_words[:1][0][:50])

['uc', 'irvine', 'uci', 'recently', 'announced', 'it', 'will', 'open', 'up', 'its', 'anthropology', 'course', 'titled', 'environmental', 'injustice', 'to', 'all', 'uc', 'students', 'during', 'the', 'upcoming', 'summer', 'and', 'fall', 'quarters', 'this', 'course', 'will', 'be', 'available', 'as', 'anth', 'and', 'will', 'focus', 'on', 'researching', 'case', 'studies', 'of', 'environmental', 'injustice', 'in', 'communities', 'across', 'california', 'the', 'bottom', 'line']


#### Creating Bigram and Trigram Models

In [23]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0][:50]]])

['uc', 'irvine', 'uci', 'recently', 'announced', 'it', 'will', 'open', 'up', 'its', 'anthropology', 'course', 'titled', 'environmental', 'injustice', 'to', 'all', 'uc', 'students', 'during', 'the', 'upcoming', 'summer', 'and', 'fall', 'quarters', 'this', 'course', 'will', 'be', 'available', 'as', 'anth', 'and', 'will', 'focus', 'on', 'researching', 'case', 'studies', 'of', 'environmental', 'injustice', 'in', 'communities', 'across', 'california', 'the', 'bottom_line']


Looks like Gensim's bigram/trigram model picks up 'bottom_line' as a phrase, which makes sense as the words are often stated in the paper as "The Bottom Line."

In [24]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Removing stopwords, creating bigrams, and lemmatizing the bigrams.

In [25]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:50])

['recently', 'announce', 'open', 'anthropology', 'course', 'title', 'environmental', 'student', 'upcome', 'summer', 'fall', 'quarter', 'course', 'available', 'anth', 'focus', 'research', 'case', 'study', 'environmental', 'injustice', 'community', 'tbl', 'sit', 'teach', 'assistant', 'learn', 'upcoming', 'course', 'course', 'create', 'research', 'experience', 'environmental', 'disaster', 'affect', 'fortun', 'part', 'platform', 'experimental', 'collaborative', 'ethnography', 'pece', 'open', 'source', 'software', 'bring', 'scientist', 'discipline', 'collaboration']


Creating a dictionary for the context corpus.

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:50])

[(0, 1), (1, 2), (2, 2), (3, 2), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 5), (32, 2), (33, 1), (34, 8), (35, 1), (36, 1), (37, 2), (38, 1), (39, 3), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 12), (48, 1), (49, 1)]


#### Topic modeling via Gensim LDA

Building the LDA model.

In [27]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"student" + 0.010*"university" + 0.007*"say" + 0.007*"community" + '
  '0.007*"go" + 0.006*"party" + 0.006*"year" + 0.006*"people" + '
  '0.006*"ranking" + 0.006*"make"'),
 (1,
  '0.032*"student" + 0.017*"campus" + 0.008*"fee" + 0.008*"service" + '
  '0.007*"reopen" + 0.006*"program" + 0.006*"covid" + 0.006*"undergraduate" + '
  '0.006*"quarter" + 0.006*"also"'),
 (2,
  '0.027*"student" + 0.024*"vaccine" + 0.014*"covid" + 0.012*"people" + '
  '0.012*"say" + 0.009*"health" + 0.006*"receive" + 0.006*"county" + '
  '0.006*"get" + 0.005*"many"'),
 (3,
  '0.015*"police" + 0.012*"say" + 0.012*"ordinance" + 0.012*"student" + '
  '0.011*"campus" + 0.011*"community" + 0.009*"people" + 0.008*"county" + '
  '0.008*"take" + 0.008*"cop"'),
 (4,
  '0.034*"student" + 0.010*"school" + 0.010*"program" + 0.009*"year" + '
  '0.009*"covid" + 0.008*"county" + 0.007*"make" + 0.007*"pell_grant" + '
  '0.006*"grant" + 0.006*"person"'),
 (5,
  '0.012*"group" + 0.012*"event" + 0.010*"protest" + 0.

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.433683826035965

Coherence Score:  0.3472314215321708


#### Article Title Length vs. Page Views

Short news article titles are often vague or too broad, leaving out key information that usually tells more of the specific truths of the article. Therefore, I wanted to see whether the lengths of the titles had any correlation with the page views of the articles.

In [30]:
df['Title_Length_W'] = df.apply(lambda row: len((row.Title.split())), axis=1)

fig1 = alt.Chart(df[['Title_Length_W', 'Views']]).mark_bar().encode(
    y = alt.Y('mean(Views)', title = 'Mean Count of Page Views', scale = alt.Scale(zero = False)),
    x = alt.X('Title_Length_W', title = 'Count of Words in Article Title', scale = alt.Scale(zero = False))
)
fig1

alt.Chart(...)

In [31]:
df['Title_Length_C'] = df.apply(lambda row: len((row.Title)), axis=1)

fig2 = alt.Chart(df[['Title_Length_C', 'Views']]).mark_bar().encode(
    y = alt.Y('mean(Views)', title = 'Count of Page Views', scale = alt.Scale(zero = False)),
    x = alt.X('Title_Length_C', title = 'Count of Characters in Article Title', scale = alt.Scale(zero = False))
)
fig2

alt.Chart(...)

On average, it seems that article headlines with 3, 5, 7, and 9 words get the highest number of page views. There does seem to be a general increase in page views as the count of words increases, peaking at 9 words and going mostly downhill from there.

---

## Methods

### Headline vs. Content Analysis

First, I process the article text, performing tokenization and lemmatization as well as filtering out stopwords and punctuation.

Using spaCy's named entity recognition, I extract the main entity keywords from each article. Then, I compare the keywords extracted with the keywords in the headline, checking how many named entities are named in the actual headline.

I also create a weighted version of this keyword extraction and comparison, placing more weight on the keywords with higher occurrences in the article.

Finally, I used Pearson's Correlation Coefficient to determine whether the number of shared keywords between an article and its headline is positively or negatively correlated with page views.

In [32]:
def spacy_process(text):
    # lowering case messes with the entity recognition
    doc = nlp(text)
    
#Tokenization and lemmatization are done with the spacy nlp pipeline commands
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
            
    return(" ".join(filtered_sentence))

In [33]:
df['Processed_Content'] = df.apply(lambda row: spacy_process(row.Content), axis=1) # Entities is a list of spaCy spans(tokens within)

In [34]:
df.head()

,Title,Content,Views,Title_Length_W,Title_Length_C,Processed_Content
0,UC Irvine Offers UC-Wide Environmental Injusti...,UC Irvine (UCI) recently announced it will ope...,616,10,79,UC Irvine ( UCI ) recently announce open anthr...
1,UC Santa Barbara Opens Clinic to Distribute Jo...,As COVID-19 vaccines become more widely availa...,701,13,88,covid-19 vaccine widely available UC Santa Bar...
2,AS Passes Bill to Reform the Honorarium of Sen...,The UC Santa Barbara (UCSB) Associated Student...,1097,11,60,UC Santa Barbara ( UCSB ) Associated Students ...
3,Cops Off Campus: A March to Recognize the Move...,"On May 17, UC Santa Barbara’s (UCSB) Cops Off ...",748,9,50,17 UC Santa Barbara ( UCSB ) cop Campus hold a...
4,Wealthy Countries are Hoarding COVID-19 Vaccin...,If there is one thing to be learned from this ...,1458,12,81,thing learn pandemic year half ’ health care i...


#### Summarization & Headline Generation

(ended up not using this, so it's mostly just exploration)

In [35]:
nlp = spacy.load('en_core_web_md') # reload the language module (not sure why the .ents doesn't work without reloading)

In [36]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [37]:
stopwords=list(STOP_WORDS)
punctuation=punctuation+ '\n'

In [38]:
def generate_summary(text):
    doc= nlp(text)
    tokens=[token.text for token in doc]
    
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(1)
    select_length
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    final_summary
    summary=''.join(final_summary)
    return(summary)

In [39]:
df['Generated_Headline'] = df.apply(lambda row: generate_summary(row.Content), axis=1) # Entities is a list of spaCy spans(tokens within)

In [40]:
df.head()

,Title,Content,Views,Title_Length_W,Title_Length_C,Processed_Content,Generated_Headline
0,UC Irvine Offers UC-Wide Environmental Injusti...,UC Irvine (UCI) recently announced it will ope...,616,10,79,UC Irvine ( UCI ) recently announce open anthr...,\n\nStudents also cover a variety of concepts ...
1,UC Santa Barbara Opens Clinic to Distribute Jo...,As COVID-19 vaccines become more widely availa...,701,13,88,covid-19 vaccine widely available UC Santa Bar...,They sent their mobile vaccine clinic to our f...
2,AS Passes Bill to Reform the Honorarium of Sen...,The UC Santa Barbara (UCSB) Associated Student...,1097,11,60,UC Santa Barbara ( UCSB ) Associated Students ...,"\n\n""AN HONORARIUM IS THE SALARY SENATORS\n\n ..."
3,Cops Off Campus: A March to Recognize the Move...,"On May 17, UC Santa Barbara’s (UCSB) Cops Off ...",748,9,50,17 UC Santa Barbara ( UCSB ) cop Campus hold a...,\r\n\r\nMany local I.V. residents joined the m...
4,Wealthy Countries are Hoarding COVID-19 Vaccin...,If there is one thing to be learned from this ...,1458,12,81,thing learn pandemic year half ’ health care i...,\r\n\r\nIn an interview with The Bottom Line (...


#### Named Entity Recognition

In [41]:
def get_ents(text):  
    return list((nlp(text)).ents)

In [42]:
df['Entities'] = df.apply(lambda row: get_ents(row.Processed_Content), axis=1) # Entities is a list of spaCy spans(tokens within)
df['Entities'] = df.apply(lambda row: [span.text for span in row.Entities], axis=1)

In [43]:
df.head()

,Title,Content,Views,Title_Length_W,Title_Length_C,Processed_Content,Generated_Headline,Entities
0,UC Irvine Offers UC-Wide Environmental Injusti...,UC Irvine (UCI) recently announced it will ope...,616,10,79,UC Irvine ( UCI ) recently announce open anthr...,\n\nStudents also cover a variety of concepts ...,"[UC Irvine, UCI, Environmental Injustice UC, s..."
1,UC Santa Barbara Opens Clinic to Distribute Jo...,As COVID-19 vaccines become more widely availa...,701,13,88,covid-19 vaccine widely available UC Santa Bar...,They sent their mobile vaccine clinic to our f...,"[covid-19, UC Santa Barbara, UCSB, Santa Barba..."
2,AS Passes Bill to Reform the Honorarium of Sen...,The UC Santa Barbara (UCSB) Associated Student...,1097,11,60,UC Santa Barbara ( UCSB ) Associated Students ...,"\n\n""AN HONORARIUM IS THE SALARY SENATORS\n\n ...","[UC Santa Barbara, UCSB, Senate, Esmeralda Qui..."
3,Cops Off Campus: A March to Recognize the Move...,"On May 17, UC Santa Barbara’s (UCSB) Cops Off ...",748,9,50,17 UC Santa Barbara ( UCSB ) cop Campus hold a...,\r\n\r\nMany local I.V. residents joined the m...,"[17, UC, Santa Barbara, UCSB, UCSB, Black Brow..."
4,Wealthy Countries are Hoarding COVID-19 Vaccin...,If there is one thing to be learned from this ...,1458,12,81,thing learn pandemic year half ’ health care i...,\r\n\r\nIn an interview with The Bottom Line (...,"[covid-19, 80 percent, 0.2 percent, 46.04 perc..."


#### Comparing Number of Keyword (Named Entity) Matches in Title

- Convert the list of entities (which are spaCy spans) to a simple set
- Check for the number of shared words
- For later considerations:
    - Set frequency of the items of the list as a count
    - Add up the counts for the entities so that words that occur more often have a higher weight

In [44]:
import re

In [45]:
def get_shared_entities(title, entities):
    shared_entities = []
    title = spacy_process(title).lower()
    entities_set = set(entities)
    for entity in entities_set:
        try:
            word_re = re.compile(entity.lower())
            if word_re.search(title):
                shared_entities.append(entity)
        except:
            return (shared_entities)
    return (shared_entities)

In [46]:
get_shared_entities(df.Title[0], df.Entities[0])

['UC Irvine', 'summer', 'UC']

In [47]:
df['Entity_Count'] = df.apply(lambda row: len(get_shared_entities(row['Title'], row['Entities'])), axis=1)

In [48]:
df.head()

,Title,Content,Views,Title_Length_W,Title_Length_C,Processed_Content,Generated_Headline,Entities,Entity_Count
0,UC Irvine Offers UC-Wide Environmental Injusti...,UC Irvine (UCI) recently announced it will ope...,616,10,79,UC Irvine ( UCI ) recently announce open anthr...,\n\nStudents also cover a variety of concepts ...,"[UC Irvine, UCI, Environmental Injustice UC, s...",3
1,UC Santa Barbara Opens Clinic to Distribute Jo...,As COVID-19 vaccines become more widely availa...,701,13,88,covid-19 vaccine widely available UC Santa Bar...,They sent their mobile vaccine clinic to our f...,"[covid-19, UC Santa Barbara, UCSB, Santa Barba...",5
2,AS Passes Bill to Reform the Honorarium of Sen...,The UC Santa Barbara (UCSB) Associated Student...,1097,11,60,UC Santa Barbara ( UCSB ) Associated Students ...,"\n\n""AN HONORARIUM IS THE SALARY SENATORS\n\n ...","[UC Santa Barbara, UCSB, Senate, Esmeralda Qui...",1
3,Cops Off Campus: A March to Recognize the Move...,"On May 17, UC Santa Barbara’s (UCSB) Cops Off ...",748,9,50,17 UC Santa Barbara ( UCSB ) cop Campus hold a...,\r\n\r\nMany local I.V. residents joined the m...,"[17, UC, Santa Barbara, UCSB, UCSB, Black Brow...",0
4,Wealthy Countries are Hoarding COVID-19 Vaccin...,If there is one thing to be learned from this ...,1458,12,81,thing learn pandemic year half ’ health care i...,\r\n\r\nIn an interview with The Bottom Line (...,"[covid-19, 80 percent, 0.2 percent, 46.04 perc...",2


#### Comparing Number of Weighted Keyword (Named Entity) Matches in Title

In [49]:
from collections import Counter

In [50]:
def get_weighted_shared_entities_count(title, entities):
    count = 0
    entities_counter = Counter(entities)
    shared_entities = get_shared_entities(title, entities)
    
    for entity in shared_entities:
        count += entities_counter[entity]
    
    return count

In [51]:
df['Entity_Weighted_Count'] = df.apply(lambda row: get_weighted_shared_entities_count(row['Title'], row['Entities']), axis=1)

In [52]:
df.head()

,Title,Content,Views,Title_Length_W,Title_Length_C,Processed_Content,Generated_Headline,Entities,Entity_Count,Entity_Weighted_Count
0,UC Irvine Offers UC-Wide Environmental Injusti...,UC Irvine (UCI) recently announced it will ope...,616,10,79,UC Irvine ( UCI ) recently announce open anthr...,\n\nStudents also cover a variety of concepts ...,"[UC Irvine, UCI, Environmental Injustice UC, s...",3,6
1,UC Santa Barbara Opens Clinic to Distribute Jo...,As COVID-19 vaccines become more widely availa...,701,13,88,covid-19 vaccine widely available UC Santa Bar...,They sent their mobile vaccine clinic to our f...,"[covid-19, UC Santa Barbara, UCSB, Santa Barba...",5,9
2,AS Passes Bill to Reform the Honorarium of Sen...,The UC Santa Barbara (UCSB) Associated Student...,1097,11,60,UC Santa Barbara ( UCSB ) Associated Students ...,"\n\n""AN HONORARIUM IS THE SALARY SENATORS\n\n ...","[UC Santa Barbara, UCSB, Senate, Esmeralda Qui...",1,3
3,Cops Off Campus: A March to Recognize the Move...,"On May 17, UC Santa Barbara’s (UCSB) Cops Off ...",748,9,50,17 UC Santa Barbara ( UCSB ) cop Campus hold a...,\r\n\r\nMany local I.V. residents joined the m...,"[17, UC, Santa Barbara, UCSB, UCSB, Black Brow...",0,0
4,Wealthy Countries are Hoarding COVID-19 Vaccin...,If there is one thing to be learned from this ...,1458,12,81,thing learn pandemic year half ’ health care i...,\r\n\r\nIn an interview with The Bottom Line (...,"[covid-19, 80 percent, 0.2 percent, 46.04 perc...",2,4


## Results

### Searching for correlation via linear regression

#### Number of Keyword Matches and Page Views

In [53]:
scatter_entcountviews = alt.Chart(df[['Entity_Count', 'Views']]).mark_circle().encode(
    y = alt.Y('Views', title = 'Article Page Views', scale = alt.Scale(zero = False)),
    x = alt.X('Entity_Count', title = 'Number of Shared Entities in Headline', scale = alt.Scale(zero = False))
)
scatter_entcountviews + scatter_entcountviews.transform_regression('Entity_Count', 'Views').mark_line()

alt.LayerChart(...)

Pearson correlation coefficient: 0.13352267, meaning very low positive correlation

In [54]:
np.corrcoef(df.Entity_Count, df.Views)

array([[1.        , 0.17530516],
       [0.17530516, 1.        ]])

### Number of Weighted Keyword Matches and Page Views

In [55]:
scatter_weightentcountviews = alt.Chart(df[['Entity_Weighted_Count', 'Views']]).mark_circle().encode(
    y = alt.Y('Views', title = 'Article Page Views', scale = alt.Scale(zero = False)),
    x = alt.X('Entity_Weighted_Count', title = 'Weighted Number of Shared Entities in Headline', scale = alt.Scale(zero = False))
)
scatter_weightentcountviews + scatter_weightentcountviews.transform_regression('Entity_Weighted_Count', 'Views').mark_line()

alt.LayerChart(...)

Pearson correlation coefficient: 0.31823466, meaning low positive correlation

In [56]:
np.corrcoef(df.Entity_Weighted_Count, df.Views)

array([[1.        , 0.34506912],
       [0.34506912, 1.        ]])